# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, Annotated[List[float], Len(min_length=2, max_length=2)]] = {}, constraints: Dict[str, Annotated[List[Union[float, xopt.vocs.ConstraintEnum]], Len(min_length=2, max_length=2)]] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, observables: List[str] = []) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that va

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [4]:
# as dict
vocs.dict()

/tmp/ipykernel_2801/20160725.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  vocs.dict()


{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

/tmp/ipykernel_2801/3773362369.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  vocs2 = VOCS.parse_obj(vocs.dict())
/tmp/ipykernel_2801/3773362369.py:2: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  vocs2 = VOCS.parse_obj(vocs.dict())


In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

/tmp/ipykernel_2801/1166166515.py:2: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  vocs.json()


'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,695.565235,0.535904,1234,696.101139,695.029331,1392.202278,1390.058661
6,229.777907,-0.839272,1234,228.938635,230.617180,457.877270,461.234359
7,426.673302,-0.027573,1234,426.645730,426.700875,853.291460,853.401750
8,856.239440,0.200815,1234,856.440255,856.038626,1712.880510,1712.077252
9,104.968966,-0.969844,1234,103.999122,105.938811,207.998244,211.877621
10,817.272395,0.535278,1234,817.807673,816.737116,1635.615346,1633.474233
11,723.964763,-0.870283,1234,723.094480,724.835046,1446.188960,1449.670091
12,823.192521,0.720923,1234,823.913444,822.471598,1647.826888,1644.943196
13,414.793922,-0.687150,1234,414.106772,415.481073,828.213544,830.962145
14,895.058281,-0.697968,1234,894.360313,895.756249,1788.720626,1791.512498


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-696.101139,695.029331
6,-228.938635,230.617180
7,-426.645730,426.700875
8,-856.440255,856.038626
9,-103.999122,105.938811
10,-817.807673,816.737116
11,-723.094480,724.835046
12,-823.913444,822.471598
13,-414.106772,415.481073
14,-894.360313,895.756249


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-696.101139,695.029331
6,-228.938635,230.617180
7,-426.645730,426.700875
8,-856.440255,856.038626
9,-103.999122,105.938811
10,-817.807673,816.737116
11,-723.094480,724.835046
12,-823.913444,822.471598
13,-414.106772,415.481073
14,-894.360313,895.756249


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-696.10113881,  695.02933063],
       [-228.9386348 ,  230.61717951],
       [-426.64572976,  426.7008752 ],
       [-856.44025514,  856.03862586],
       [-103.99912214,  105.93881068],
       [-817.8076732 ,  816.73711646],
       [-723.09447998,  724.83504564],
       [-823.9134438 ,  822.4715979 ],
       [-414.10677225,  415.48107261],
       [-894.36031289,  895.75624894]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1390.202278,-1390.058661
6,455.877270,-461.234359
7,851.291460,-853.401750
8,1710.880510,-1712.077252
9,205.998244,-211.877621
10,1633.615346,-1633.474233
11,1444.188960,-1449.670091
12,1645.826888,-1644.943196
13,826.213544,-830.962145
14,1786.720626,-1791.512498


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
